<a href="https://colab.research.google.com/github/novelworm777/bangkit-kaidah-machine-learning/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# only run this in google colab to upload data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving jdih_kemenkeu_regulations.csv to jdih_kemenkeu_regulations.csv
User uploaded file "jdih_kemenkeu_regulations.csv" with length 1537224 bytes


In [68]:
import pandas as pd

raw_df = pd.read_csv('jdih_kemenkeu_regulations.csv')
print(raw_df.shape)
raw_df.head(10)

(9176, 2)


,title,label
0,Jenis dan Tarif atas Jenis Penerimaan Negara B...,"Jenis Dan Tarif, Penerimaan Negara Bukan Pajak..."
1,"Tata Cara Pendaftaran, Pelaporan, dan Pendataa...","Pajak Bumi Dan Bangunan, Pendaftaran Dan Pelap..."
2,Pedoman Standar Pelayanan di Lingkungan Kement...,"Pedoman, Standar Pelayanan, Kementerian Keuangan"
3,Account Representative pada Kantor Pelayanan P...,"Account Representative, Kantor Pelayanan Pajak"
4,Perubahan Kedua atas Peraturan Menteri Keuanga...,"Perubahan Kedua, Program Pen, Pemerintah Daerah"
5,Pemberian Tunjangan Hari Raya dan Gaji Ketiga ...,"Tunjangan Hari Raya, Gaji Ketiga Belas, Tahun ..."
6,Petunjuk Teknis Pelaksanaan Pemberian Tunjanga...,"Petunjuk Teknis, Tunjangan Hari Raya, Gaji Ket..."
7,Tata Cara Penundaan dan/ atau Pemotongan Dana ...,"Dana Desa, Dana Perimbangan, Pemotongan"
8,Perubahan atas Peraturan Menteri Keuangan Nomo...,"Perubahan, Program Pen, Tagihan Listrik"
9,Biaya Operasional Penyelenggaraan Pembayaran M...,"Biaya Operasional, Pembayaran Manfaat Pensiun,..."


In [19]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9176 entries, 0 to 9175
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9170 non-null   object
 1   label   9176 non-null   object
dtypes: object(2)
memory usage: 143.5+ KB


In [21]:
# drop null rows
dropped_df = raw_df.dropna()
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9170 entries, 0 to 9175
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9170 non-null   object
 1   label   9170 non-null   object
dtypes: object(2)
memory usage: 214.9+ KB


In [66]:
import os

def SortLabel(x):
    x = x.split(', ')
    x.sort()
    return ', '.join(x)

# sort labels
sorted_df['label'] = dropped_df['label'].apply(SortLabel)
sorted_df

,title,label
9036,Acara Penetapan Ganti Kerugian oleh Pengadilan...,"Hak Atas Tanah, Penetapan, Pengadilan Tinggi"
3,Account Representative pada Kantor Pelayanan P...,"Account Representative, Kantor Pelayanan Pajak"
1747,Account Representative pada Kantor Pelayanan P...,"Account Representative, Kantor Pelayanan Pajak"
1970,Administrasi Pemerintahan.,Administrasi Pemerintahan
1097,Administrasi Pengelolaan Hibah.,"Administrasi, Hibah, Pengelolaan Pembiayaan"
...,...,...
5496,"Wewenang Pengangkatan, Pemindahan, dan Pemberh...","Pegawai Negeri Sipil, Pengangkatan, Wewenang"
8857,Wewenang Penggunaan Consular Fee,"Consular Fee, Wewenang"
4342,Wilayah Negara.,Wilayah Negara
5639,Yayasan.,Yayasan


In [69]:
# drop duplicate rows (title and label)
drop_duplicated_df = sorted_df.copy()
drop_duplicated_df.drop_duplicates(keep='first', inplace=True)
drop_duplicated_df

,title,label
9036,Acara Penetapan Ganti Kerugian oleh Pengadilan...,"Hak Atas Tanah, Penetapan, Pengadilan Tinggi"
3,Account Representative pada Kantor Pelayanan P...,"Account Representative, Kantor Pelayanan Pajak"
1747,Account Representative pada Kantor Pelayanan P...,"Account Representative, Kantor Pelayanan Pajak"
1970,Administrasi Pemerintahan.,Administrasi Pemerintahan
1097,Administrasi Pengelolaan Hibah.,"Administrasi, Hibah, Pengelolaan Pembiayaan"
...,...,...
5496,"Wewenang Pengangkatan, Pemindahan, dan Pemberh...","Pegawai Negeri Sipil, Pengangkatan, Wewenang"
8857,Wewenang Penggunaan Consular Fee,"Consular Fee, Wewenang"
4342,Wilayah Negara.,Wilayah Negara
5639,Yayasan.,Yayasan


In [72]:
drop_duplicated_df[drop_duplicated_df.duplicated(['title'], keep='first')]

,title,label
3981,Alokasi Kurang Bayar Dana Bagi Hasil Sumber Da...,"Alokasi Kurang Bayar, Dana Bagi Hasil, Minyak ..."
3794,Batasan Kegiatan dan Jenis Jasa Kena Pajak yan...,"Batasan Kegiatan, Jenis Jasa Kena Pajak, Pajak..."
2911,Batasan dan Tata Cara Pengenaan Pajak Pertamba...,"Batasan Dan Tata Cara, Kegiatan Membangun Send..."
1719,Bentuk dan Mekanisme Pendanaan Perguruan Tingg...,"Pendanaan, Perguruan Tinggi Negeri Badan Hukum"
1943,Dana Perhitungan Fihak Ketiga.,"Dana, Perhitungan, Pihak Ketiga"
...,...,...
6837,Tempat Penimbunan Berikat.,"Penimbunan Berikat, Tempat"
6874,Tidak Dipungut Cukai.,"Barang Kena Cukai, Tidak Dipungut"
2746,Transaksi Lindung Nilai dalam Pengelolaan Utan...,"Pengelolaan, Transaksi Lindung Nilai, Utang Pe..."
2161,Transaksi Surat Utang Negara Secara Langsung.,"Pengelolaan Pembiayaan, Surat Utang Negara, Tr..."


1. drop null
2. labels to columns
3. labels with less than X data will be 'other'
4. sort by column other (ascending) (https://stackoverflow.com/questions/17141558/how-to-sort-a-dataframe-in-python-pandas-by-two-or-more-columns)
5. drop duplicated title rows (keep='first') (https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/)